In [2]:
import sys
import os
import yaml
import pandas as pd
import numpy as np
import os
import glob
import pickle
import logging
sys.path.append('./..')
sys.path.append('./../..')
import multiprocessing as mp

import torch
import torch.nn as nn
import torch.nn.functional as F
from random import shuffle

print( torch.cuda.is_available(), torch.cuda.current_device(),torch.cuda.get_device_name(0))
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"

device = torch.device(dev)


try:
    from . import get_embeddings
    from . import context_vector_model_1 as c2v
    from .src.data_fetcher import data_fetcher
    from . import utils_1
except:
    import get_embeddings
    import context_vector_model_1 as c2v
    from src.data_fetcher import data_fetcher
    import utils_1


True 0 Tesla P100-PCIE-16GB


/home/ddatta/anaconda3/envs/AD_v2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ddatta/anaconda3/envs/AD_v2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ddatta/anaconda3/envs/AD_v2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ddatta/anaconda3/envs/AD_v2/lib/python3.6/s

In [3]:
CONFIG_FILE = 'config_1.yaml'
DIR = None
OP_DIR = None
modelData_SaveDir = None
DATA_DIR = None
num_jobs = None
CONFIG = None
Refresh_Embeddings = None
logger = None
domain_dims = None

# ------ #

def get_domain_dims(dd_file_path):
    with open(dd_file_path, 'rb') as fh:
        domain_dims = pickle.load(fh)
    _tmpDF = pd.DataFrame.from_dict(domain_dims, orient='index')
    _tmpDF = _tmpDF.reset_index()
    _tmpDF = _tmpDF.rename(columns={'index': 'domain'})
    _tmpDF = _tmpDF.sort_values(by=['domain'])
    res = {k: v for k, v in zip(_tmpDF['domain'], _tmpDF[0])}
    return res


def setup_config(_DIR=None):
    global CONFIG_FILE
    global DATA_DIR
    global modelData_SaveDir
    global OP_DIR
    global DIR
    global num_jobs
    global Refresh_Embeddings
    global logger
    global CONFIG
    global domain_dims

    with open(CONFIG_FILE) as f:
        CONFIG = yaml.safe_load(f)
    if _DIR is None:
        DIR = CONFIG['DIR']
    else:
        DIR = _DIR

    DATA_DIR = os.path.join(CONFIG['DATA_DIR'])
    modelData_SaveDir = os.path.join(
        CONFIG['model_data_save_dir'],
        DIR
    )

    if not os.path.exists(CONFIG['OP_DIR']):
        os.mkdir(CONFIG['OP_DIR'])
    OP_DIR = os.path.join(CONFIG['OP_DIR'], DIR)
    if not os.path.exists(OP_DIR):
        os.mkdir(OP_DIR)

    Refresh_Embeddings = CONFIG[DIR]['Refresh_Embeddings']
    cpu_count = mp.cpu_count()
    num_jobs = min(cpu_count, CONFIG['num_jobs'])

    if not os.path.exists(CONFIG['model_data_save_dir']):
        os.mkdir(CONFIG['model_data_save_dir'])

    if not os.path.exists(modelData_SaveDir):
        os.mkdir(modelData_SaveDir)
    
    domain_dims_file = os.path.join(DATA_DIR, DIR, "domain_dims.pkl")
    domain_dims = get_domain_dims(domain_dims_file)
    print(' Set up config')
    return

setup_config()

 Set up config


In [4]:
def get_training_data():
    global DATA_DIR
    global DIR

    train_x_pos, train_x_neg, _, _, _, _ = data_fetcher.get_data_MEAD(
        DATA_DIR,
        DIR,
        anomaly_type=1
    )
    train_x_pos = np.reshape(train_x_pos, [-1, train_x_pos.shape[1], 1])
    train_x_neg = np.reshape(train_x_neg, [-1, train_x_neg.shape[1], train_x_neg.shape[2], 1])
    return train_x_pos, train_x_neg

In [5]:
def custom_loss(y_pred, y_true):
    loss = torch.mean(y_pred)
    return loss

In [6]:
class Net(nn.Module):

    def __init__(self):
        # this is the place where you instantiate all your modules
        # you can later access them using the same names you've given them in
        # here
        super(Net, self).__init__()
        
        
    def setup_model(
        self, 
        emb_dim,
        domain_dims,
        domain_emb_wt,
        lstm_dim,
        interaction_layer_dim,
        intermediate_context_dims,
        num_neg_samples,
        domain_emb_wt_list = None
    ):
        
        self.num_domains = len(domain_dims)
        self.domain_dims = domain_dims
        self.lstm_dim = lstm_dim
        self.num_neg_samples = num_neg_samples
        self.list_Entity_Embed = []
        self.interaction_layer_dim = interaction_layer_dim
        
        for d_idx in range(self.num_domains):
            # e = nn.Embedding(num_embeddings= self.domain_dims[d_idx], embedding_dim=emb_dim)
            weight = domain_emb_wt[d_idx]
            e = nn.Embedding.from_pretrained(weight)
            self.list_Entity_Embed.append(e)
        
        self.BD_LSTM_layer = nn.LSTM(
            input_size = emb_dim,
            hidden_size = lstm_dim,
            num_layers = 1,
            batch_first = True,
            bidirectional = True
        )
        
        self.FNN_1 = nn.Linear(self.lstm_dim*2, intermediate_context_dims[0])
        self.FNN_2 = nn.Linear(intermediate_context_dims[0], self.interaction_layer_dim) 
        self.FNN_3 = nn.Linear(emb_dim, self.interaction_layer_dim) 
        return
    
    
    # Define network structure
    def forward(self, input_x_pos, input_x_neg=None):
        
         # ------------- main function ------------------- #
        def process(x,_type='pos'):
            
            split_x = torch.split(
                x, 
                split_size_or_sections = 1, 
                dim=1
            )
            # ------------
            # emb_op should be [ ?, num_domains, emb_dim]
            emb_op = []
            for i in range(self.num_domains) :
                _emb_layer = self.list_Entity_Embed[i]
                _x =  torch.squeeze(split_x[i],dim=-1)
                emb_op.append(_emb_layer(_x))
            
            # ------------
            # Calculate mean of the embeddings
            stacked = torch.squeeze(torch.stack( emb_op, dim = 1),dim=-2)
            _mean = torch.mean(stacked, dim=1, keepdim=True, out=None)
            padded_seq = torch.cat([_mean, stacked, _mean], dim=1)
                      
            lstm_out, lstm_hidden = self.BD_LSTM_layer(padded_seq)
            # break up lstm output 
            
            n_timesteps =  self.num_domains + 2
            _parts = torch.chunk(lstm_out, 2, dim=-1) 
            
            bdLstm_fwd = torch.chunk(_parts[0], n_timesteps, dim=1)
            bdLstm_rev = torch.chunk(_parts[1], n_timesteps, dim=1)       
            
            # ------------
            op_list = []
            for i in range(1, n_timesteps - 1):
                idx = i-1
                left_idx = i-1
                right_idx = i+1
                _left = bdLstm_fwd[left_idx]
                _right = bdLstm_rev[right_idx]
                # shape should be [?, 2*lstm_dim]
                ctx_concat = torch.squeeze(torch.cat([_left, _right],dim=-1),dim=1)
                ctx = self.FNN_1(ctx_concat)
                ctx = self.FNN_2(ctx)
             
                # Now, transform the input embedding
                xformed_inp = self.FNN_3(emb_op[idx])
                
                # Calculate dot product
                _dim = self.interaction_layer_dim
                dot_product = torch.bmm(
                    ctx.view([-1,1,_dim]) ,  xformed_inp.view([-1,_dim,1])
                )
                _op = torch.squeeze(dot_product,dim=-1)
                op_list.append(_op)
               
                
            cur_op = torch.squeeze(torch.stack( op_list, dim=1), dim=-1 )
            cur_op = torch.sum( cur_op, dim=-1, keepdim=True)
            if _type == 'neg':
                cur_op = -cur_op
            
            cur_op = torch.sigmoid(cur_op)         
            return cur_op
        
        # ------------
        
        if input_x_neg is None:
            return process(input_x_pos)
            # Mean so that gradients can be calculated
        else:
            pos = torch.log(process(input_x_pos))
            list_neg_x = torch.chunk(
                input_x_neg, 
                self.num_neg_samples, 
                dim = 1
            )
            
            list_neg_x = [ torch.squeeze(_,dim=1) for _ in list_neg_x]
            ns = [] 
            
            for _neg_x in list_neg_x:    
                tmp = torch.log(process(_neg_x,'neg'))
                ns.append(tmp)
                
            ns = torch.stack(ns,dim=1)
            neg = torch.mean(ns)
            
            # -----------------
            # This is the objective function 
            # -----------------
            res = pos + neg
            # maximize P, means minimize -P
            return -res
          

In [9]:
def get_entity_embeddings():
    global CONFIG
    global DIR
    global DATA_DIR
    global modelData_SaveDir

    embedding_dims = CONFIG[DIR]['entity_embedding_dims']
    eEmb_num_epochs = CONFIG[DIR]['eEmb_num_epochs']

    # ============== ENTITY EMBEDDING ================ #
    # -------------------------------------------------
    # Check if files exist , if not generate embeddings
    # -------------------------------------------------
    training_data_file = CONFIG['train_data_file']
    files_exist = len(glob.glob(os.path.join(modelData_SaveDir, 'init_embedding**.npy'))) > 0
    if not files_exist or Refresh_Embeddings:
        src_DIR = os.path.join(DATA_DIR, DIR)
        get_embeddings.get_initial_entity_embeddings(
            training_data_file,
            modelData_SaveDir,
            src_DIR,
            embedding_dims,
            eEmb_num_epochs
        )

    # ----- Read in domain_embedding weights ------- #

    domain_emb_wt = []
    file_list = sorted(glob.glob(
        os.path.join(modelData_SaveDir, 'init_embedding**{}.npy'.format(embedding_dims))))
    for npy_file in file_list:
        _tmp_ = np.load(npy_file)
        domain_emb_wt.append(_tmp_)

    return domain_emb_wt

In [7]:
class model():
    
    def __init__(self):
        self.net = None
        return
        
        
    def set_hyperparams(
        self,
        emb_dim,
        domain_emb_wt,
        domain_dims,
        lstm_dim,
        interaction_layer_dim=32,
        intermediate_context_dims=[64],
        num_neg_samples = 15,
        model_signature = None,
        save_dir = None,
        num_epochs=5,
        batch_size=256
    ):
        
        self.domain_emb_wt = domain_emb_wt
        self.emb_dim=emb_dim
        self.domain_dims=domain_dims
        self.lstm_dim=lstm_dim
        self.interaction_layer_dim=interaction_layer_dim
        self.intermediate_context_dims = intermediate_context_dims
        self.num_neg_samples=num_neg_samples
        self.model_signature=model_signature
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        
        self.save_dir = save_dir
        self.model_signature = model_signature
        self.model_file_name = self.model_signature + ".pkl"
        self.model_weights_path = os.path.join(self.save_dir, self.model_file_name)
        self.log_interval = 100
        
    def build_model(self):
        
        self.net = Net()
        
        self.net.setup_model( 
            emb_dim = self.emb_dim,
            domain_dims = self.domain_dims,
            domain_emb_wt =self.domain_emb_wt,
            lstm_dim = self.lstm_dim,
            interaction_layer_dim = self.interaction_layer_dim,
            intermediate_context_dims = self.intermediate_context_dims,
            num_neg_samples = self.num_neg_samples
        )
        print(self.net)
        
        self.optimizer = torch.optim.Adam(self.net.parameters(), lr=0.0025)
        self.criterion = custom_loss
        
        # ================== 
    
    def train_model(self, train_x_pos, train_x_neg):
        
        data_len = train_x_pos.shape[0]
        num_batches = data_len // self.batch_size
        bs = self.batch_size
        log_interval = 100
        
        for epoch in range(self.num_epochs):
    
            # Shuffle
            ind_list = list(range(train_x_neg.shape[0]))
            shuffle(ind_list)
            _train_x_pos = train_x_pos[ind_list, :, :]
            _train_x_neg = train_x_neg[ind_list, : , : , :]
    
            for batch_idx in range(num_batches+1):

                _x_pos = _train_x_pos[batch_idx*bs:(batch_idx+1)*bs]
                _x_neg = _train_x_neg[batch_idx*bs:(batch_idx+1)*bs]

                # feed tensor
                _x_pos = torch.LongTensor(_x_pos)
                _x_neg = torch.LongTensor(_x_neg)

                # ----- #
                self.optimizer.zero_grad()
                output = self.net(_x_pos,_x_neg)
                loss = self.criterion(output, None)
                loss.backward()
                self.optimizer.step()
                # ----- #
                if batch_idx % log_interval == 0:
                     print('Train ::  Epoch: {}, Batch {}, Loss {:4f}'.format(epoch, batch_idx,loss))
                        
        # save model
        torch.save(self.net.state_dict(), self.model_weights_path)
        return

    
    def load_model(self):
            
        self.build_model()
        self.net.load_state_dict(torch.load(self.model_weights_path))
        self.net.eval()
        return 


In [10]:
# ------------------ #
train_x_pos, train_x_neg = get_training_data()
domain_dim_vals= list(domain_dims.values())
model_signature = 'model_003'
domain_emb_wt = get_entity_embeddings()
domain_emb_wt = [torch.FloatTensor(_) for _ in domain_emb_wt]

In [32]:
# x = train_x_pos [:100]
# x = torch.LongTensor(x)
# x.shape
# output = net(x)
# loss = custom_loss(output, None)
# loss.backward()
# x_neg = train_x_neg[:100]
# x_neg = torch.LongTensor(x_neg)
# x_neg.shape
# output = net(x,x_neg)
# loss = custom_loss(output, None)
# loss.backward()

In [11]:
model_obj = model()
model_obj.set_hyperparams(
        emb_dim = 256,
        domain_dims = domain_dim_vals,
        domain_emb_wt = domain_emb_wt,
        lstm_dim=64,
        interaction_layer_dim=32,
        intermediate_context_dims=[48],
        num_epochs=10,
        batch_size=512,
        model_signature=model_signature,
        save_dir=modelData_SaveDir
    )

model_obj.build_model()





Net(
  (BD_LSTM_layer): LSTM(256, 64, batch_first=True, bidirectional=True)
  (FNN_1): Linear(in_features=128, out_features=48, bias=True)
  (FNN_2): Linear(in_features=48, out_features=32, bias=True)
  (FNN_3): Linear(in_features=256, out_features=32, bias=True)
)


In [35]:
model_obj.train_model(train_x_pos, train_x_neg)

Train ::  Epoch: 0, Batch 0, Loss 1.546731
Train ::  Epoch: 0, Batch 100, Loss 1.014935
Train ::  Epoch: 0, Batch 200, Loss 0.727491
Train ::  Epoch: 1, Batch 0, Loss 0.700407
Train ::  Epoch: 1, Batch 100, Loss 0.648676
Train ::  Epoch: 1, Batch 200, Loss 0.697568
Train ::  Epoch: 2, Batch 0, Loss 0.548770
Train ::  Epoch: 2, Batch 100, Loss 0.587405
Train ::  Epoch: 2, Batch 200, Loss 0.502573
Train ::  Epoch: 3, Batch 0, Loss 0.513768
Train ::  Epoch: 3, Batch 100, Loss 0.458863
Train ::  Epoch: 3, Batch 200, Loss 0.457643
Train ::  Epoch: 4, Batch 0, Loss 0.526480
Train ::  Epoch: 4, Batch 100, Loss 0.401883
Train ::  Epoch: 4, Batch 200, Loss 0.373648
Train ::  Epoch: 5, Batch 0, Loss 0.390318
Train ::  Epoch: 5, Batch 100, Loss 0.433326
Train ::  Epoch: 5, Batch 200, Loss 0.356539
Train ::  Epoch: 6, Batch 0, Loss 0.400473
Train ::  Epoch: 6, Batch 100, Loss 0.357641
Train ::  Epoch: 6, Batch 200, Loss 0.496884
Train ::  Epoch: 7, Batch 0, Loss 0.359514
Train ::  Epoch: 7, Batch 

In [36]:
model_obj.load_model()